In [ ]:
#To be able to install textpack
!conda install -y -c conda-forge sparse_dot_topn

In [ ]:
!pip install textpack

In [5]:
import pandas as pd
import numpy as np
import pickle
import annotation_functions
import connection_database
from textpack import tp

## Get emails where ID is in list of IDs for annotation (no outliers or duplicates)

In [19]:
with open('list_ids_for_annotation.pkl', 'rb') as f:
    ids_list = pickle.load(f)

In [20]:
with open('dutch_cleaned_emails_id_body_org.pkl', 'rb') as f:
    all_emails = pickle.load(f)

In [21]:
mask = all_emails['id'].isin(ids_list)
emails = all_emails.loc[mask]

## Run textpack

In [ ]:
emails = annotation_functions.group_emails(emails)

## Get customer support labels

In [28]:
with open('label_CS_actions.pkl', 'rb') as f:
    email_labels = pickle.load(f)

In [29]:
email_labels = email_labels.merge(emails[["id", "groups", "only_body"]], on="id")

#Give an id to each TextPack group
email_labels['id_group'] = email_labels.groupby("groups").grouper.group_info[0]

## Make the groups and count the labels within a group

In [253]:
# 'count' represents within a group, the number of emails with a particular label
emails_grouped = email_labels.groupby(['groups','label_modified', "id_group"]).size().to_frame('count').reset_index()

## First group: grouped with at least two different labels by the customer support

In [299]:
# Keep only if at least 2 different labels are involved for one group of emails
emails_gr_dup = emails_grouped[emails_grouped.duplicated(["id_group"], keep = False)]

# Groups where at least 2 different labels are involved for one group
unique_dup = emails_grouped[emails_grouped.duplicated(["groups"])]

In [335]:
unique_dup.to_excel("for_annotation_432.xlsx")  

## First group: after manual annotation

In [8]:
labels_430 = pd.read_excel("for_annotation_432_completed.xlsx")

In [9]:
emails_430 = email_labels[["id", "label_modified", "groups", "only_body", "id_group"]].merge(labels_430[["id_group", "manual_label"]], on = "id_group")

In [16]:
emails_430.manual_label.value_counts()

0    5365
2    3502
3    1836
1    1289
4      27
Name: manual_label, dtype: int64

In [10]:
list_ids_430 = emails_430.id.tolist()

## Second group: grouped with at least two emails + classified by the customer support by a label that is not "solved" 

In [11]:
mask = email_labels['id'].isin(list_ids_430)
to_labelize = email_labels.loc[~mask]

In [32]:
to_labelize_ = to_labelize.groupby(['groups', "id_group", "label_modified"]).size().to_frame('count').reset_index()
to_labelize_gr = to_labelize_[to_labelize_["count"] > 1]
to_excel_no_0 = to_labelize_gr[to_labelize_gr.label_modified.isin([1,2,3])]

In [40]:
to_excel_no_0.to_excel("for_annotation_200.xlsx")  

## Second group: after manual annotation

In [12]:
labels_200 = pd.read_excel("for_annotation_200_completed.xlsx")

In [13]:
emails_200 = email_labels[["id", "label_modified", "groups", "only_body", "id_group"]].merge(labels_200[["id_group", "manual_label"]], on = "id_group")

In [47]:
emails_200.manual_label.value_counts()

1    355
2    200
4      4
3      2
Name: manual_label, dtype: int64

In [14]:
emails_600 = pd.concat([emails_430, emails_200])

In [15]:
emails_600.manual_label.value_counts()

0    5365
2    3702
3    1838
1    1644
4      31
Name: manual_label, dtype: int64

In [14]:
list_ids_600 = emails_600.id.tolist()

## Third group: grouped with at least three emails + classified by the customer support as "solved"

In [15]:
mask = email_labels['id'].isin(list_ids_600)
to_labelize = email_labels.loc[~mask]

In [17]:
to_labelize_ = to_labelize.groupby(['groups', "id_group", "label_modified"]).size().to_frame('count').reset_index()
to_labelize_gr = to_labelize_[to_labelize_["count"] > 2]

In [32]:
to_labelize_gr.to_excel("for_annotation_780.xlsx")  

## Third group: after manual annotation

In [16]:
labels_780 = pd.read_excel("for_annotation_780_completed.xlsx")

In [18]:
emails_780 = email_labels[["id", "label_modified", "groups", "only_body", "id_group"]].merge(labels_780[["id_group", "manual_label"]], on = "id_group")

In [21]:
emails_780.manual_label.value_counts()

0    8873
3     776
2     592
4     308
1      17
Name: manual_label, dtype: int64

## After the 3 manual annotations

In [23]:
emails_3 = pd.concat([emails_600, emails_780])

In [24]:
emails_3.manual_label.value_counts()

0    14238
2     4294
3     2614
1     1661
4      339
Name: manual_label, dtype: int64

In [25]:
# Remove emails with label 4 = we don't know
emails_3 = emails_3[emails_3.manual_label != 4]

## Add the automatic emails found in the other annotation notebook

In [6]:
with open('list_ids_labels3.pkl', 'rb') as f:
    list_ids_label3 = pickle.load(f)

In [10]:
mask = all_emails['id'].isin(list_ids_label3)
emails_to_add = all_emails.loc[mask]

emails_to_add = emails_to_add[["id", "only_body"]]
emails_to_add["manual_label"] = 3

In [16]:
all_annotated_emails = emails_3[["id", "only_body", "manual_label"]]
all_annotated_emails = pd.concat([all_annotated_emails, emails_to_add])

In [23]:
all_annotated_emails.manual_label.value_counts()

0    14238
2     4294
3     3078
1     1661
Name: manual_label, dtype: int64

## Get translated-French and translated-English

In [ ]:
mydb = connection_database.connect_db()
cursor = mydb.cursor()

command = """SELECT emails.id, content_fr, content_en FROM emails 
WHERE content_fr IS NOT NULL"""

cursor.execute(command)
table_rows = cursor.fetchall()
emails_fr_nl = pd.DataFrame(table_rows, columns=cursor.column_names)

In [ ]:
# The last name "Meeuwsen" is translated into "seagulls" in English and "mouettes" in French.
# This re-translate seagulls and mouettes into "Meeuwsen"
emails_fr_nl["content_en"] = emails_fr_nl["content_en"].apply(annotation_functions.fix_seagulls)
emails_fr_nl["content_fr"] = emails_fr_nl["content_fr"].apply(annotation_functions.fix_mouettes)

In [ ]:
final_to_export = all_annotated_emails.merge(emails_fr_nl, on = "id")

## Save final dataset

In [ ]:
with open('data_nl_english_french_23300.pkl', 'wb') as f:
    pickle.dump(final_to_export, f)